## Objective

    --to learn more about the crew ai 'flow' option


    - Research a potential lead
    - Score them
    - Email them


In [38]:
# Warning control
import warnings
import os, yaml

warnings.filterwarnings("ignore")

from crewai import Agent, Task, Crew
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_MODEL_NAME"] = os.getenv("OPENAI_MODEL_NAME")
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")

## Tasks and Agents


In [39]:
# Define file paths for YAML configurations
files = {
    "lead_agents": "config/lead_qualification_agents.yaml",
    "lead_tasks": "config/lead_qualification_tasks.yaml",
    "email_agents": "config/email_engagement_agents.yaml",
    "email_tasks": "config/email_engagement_tasks.yaml",
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, "r") as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
lead_agents_config = configs["lead_agents"]
lead_tasks_config = configs["lead_tasks"]
email_agents_config = configs["email_agents"]
email_tasks_config = configs["email_tasks"]

## structured Pydantic output


In [40]:
from pydantic import BaseModel, Field
from typing import Dict, Optional, List, Set, Tuple


class LeadPersonalInfo(BaseModel):
    name: str = Field(..., description="The full name of the lead.")
    job_title: str = Field(..., description="The job title of the lead.")
    role_relevance: int = Field(
        ...,
        ge=0,
        le=10,
        description="A score representing how relevant the lead's role is to the decision-making process (0-10).",
    )
    professional_background: Optional[str] = Field(
        ..., description="A brief description of the lead's professional background."
    )


class CompanyInfo(BaseModel):
    company_name: str = Field(
        ..., description="The name of the company the lead works for."
    )
    industry: str = Field(
        ..., description="The industry in which the company operates."
    )
    company_size: int = Field(
        ..., description="The size of the company in terms of employee count."
    )
    revenue: Optional[float] = Field(
        None, description="The annual revenue of the company, if available."
    )
    market_presence: int = Field(
        ...,
        ge=0,
        le=10,
        description="A score representing the company's market presence (0-10).",
    )


class LeadScore(BaseModel):
    score: int = Field(
        ..., ge=0, le=100, description="The final score assigned to the lead (0-100)."
    )
    scoring_criteria: List[str] = Field(
        ..., description="The criteria used to determine the lead's score."
    )
    validation_notes: Optional[str] = Field(
        None, description="Any notes regarding the validation of the lead score."
    )


class LeadScoringResult(BaseModel):
    personal_info: LeadPersonalInfo = Field(
        ..., description="Personal information about the lead."
    )
    company_info: CompanyInfo = Field(
        ..., description="Information about the lead's company."
    )
    lead_score: LeadScore = Field(
        ..., description="The calculated score and related information for the lead."
    )

## tools


In [41]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

# Instantiate tools
serper_tool = SerperDevTool()
scraper_tool = ScrapeWebsiteTool()

## CREW 1: Lead Qualification: Agents and Tasks


In [42]:
# Creating Agents
lead_data_agent = Agent(
    config=lead_agents_config["lead_data_agent"],
    tools=[serper_tool, scraper_tool],
)

cultural_fit_agent = Agent(
    config=lead_agents_config["cultural_fit_agent"],
    tools=[serper_tool, scraper_tool],
)

scoring_validation_agent = Agent(
    config=lead_agents_config["scoring_validation_agent"],
    tools=[serper_tool, scraper_tool],
)

# Creating Tasks
lead_data_task = Task(
    config=lead_tasks_config["lead_data_collection"], agent=lead_data_agent
)

cultural_fit_task = Task(
    config=lead_tasks_config["cultural_fit_analysis"], agent=cultural_fit_agent
)

scoring_validation_task = Task(
    config=lead_tasks_config["lead_scoring_and_validation"],
    agent=scoring_validation_agent,
    context=[lead_data_task, cultural_fit_task],
    output_pydantic=LeadScoringResult,
)

# Creating Crew
lead_scoring_crew = Crew(
    agents=[lead_data_agent, cultural_fit_agent, scoring_validation_agent],
    tasks=[lead_data_task, cultural_fit_task, scoring_validation_task],
    verbose=True,
)

## CREW 2: Email Engagement Crew


In [43]:
# Creating Agents
email_content_specialist = Agent(config=email_agents_config["email_content_specialist"])

engagement_strategist = Agent(config=email_agents_config["engagement_strategist"])

# Creating Tasks
email_drafting = Task(
    config=email_tasks_config["email_drafting"], agent=email_content_specialist
)

engagement_optimization = Task(
    config=email_tasks_config["engagement_optimization"], agent=engagement_strategist
)

# Creating Crew
email_writing_crew = Crew(
    agents=[email_content_specialist, engagement_strategist],
    tasks=[email_drafting, engagement_optimization],
    verbose=True,
)

## FLOW


In [44]:
from crewai import Flow
from crewai.flow.flow import listen, start


class SalesPipeline(Flow):
    @start()
    def fetch_leads(self):
        # Pull our leads from the database
        leads = [
            {
                "lead_data": {
                    "name": "João Moura",
                    "job_title": "Director of Engineering",
                    "company": "Clearbit",
                    "email": "joao@clearbit.com",
                    "use_case": "Using AI Agent to do better data enrichment.",
                },
            },
        ]
        return leads

    @listen(fetch_leads)
    def score_leads(self, leads):
        scores = lead_scoring_crew.kickoff_for_each(leads)
        self.state["score_crews_results"] = scores
        return scores

    @listen(score_leads)
    def store_leads_score(self, scores):
        # Here we would store the scores in the database
        return scores

    @listen(score_leads)
    def filter_leads(self, scores):
        return [score for score in scores if score["lead_score"].score > 70]

    @listen(filter_leads)
    def write_email(self, leads):
        scored_leads = [lead.to_dict() for lead in leads]
        emails = email_writing_crew.kickoff_for_each(scored_leads)
        return emails

    @listen(write_email)
    def send_email(self, emails):
        # Here we would send the emails to the leads
        return emails


flow = SalesPipeline()

╭──────────────────────────────────────────────── Flow Execution ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Starting Flow Execution                                                                                        │
│  Name: SalesPipeline                                                                                            │
│  ID: 591491fa-58d3-4833-83b6-fd14ccb348ed                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Plotting the FLOW


In [45]:
flow.plot()
## stores as a html file

Plot saved as crewai_flow.html


In [46]:
import webbrowser
import os

path = os.path.abspath("crewai_flow.html")
webbrowser.open(f"file://{path}")

True

## starting Flow


In [47]:
emails = await flow.kickoff_async()

🌊 Flow: SalesPipeline
    ID: 591491fa-58d3-4833-83b6-fd14ccb348ed
└── 🧠 Starting Flow...

 Flow started with ID: 591491fa-58d3-4833-83b6-fd14ccb348ed


🌊 Flow: SalesPipeline
    ID: 591491fa-58d3-4833-83b6-fd14ccb348ed
├── 🧠 Starting Flow...
└── 🔄 Running: fetch_leads

🌊 Flow: SalesPipeline
    ID: 591491fa-58d3-4833-83b6-fd14ccb348ed
├── Flow Method Step
└── ✅ Completed: fetch_leads

🌊 Flow: SalesPipeline
    ID: 591491fa-58d3-4833-83b6-fd14ccb348ed
├── Flow Method Step
├── ✅ Completed: fetch_leads
└── 🔄 Running: score_leads

...

🌊 Flow: SalesPipeline
    ID: 591491fa-58d3-4833-83b6-fd14ccb348ed
├── Flow Method Step
├── ✅ Completed: fetch_leads
├── ✅ Completed: score_leads
├── ✅ Completed: store_leads_score
├── ✅ Completed: filter_leads
└── ✅ Completed: write_email

🌊 Flow: SalesPipeline
    ID: 591491fa-58d3-4833-83b6-fd14ccb348ed
├── Flow Method Step
├── ✅ Completed: fetch_leads
├── ✅ Completed: score_leads
├── ✅ Completed: store_leads_score
├── ✅ Completed: filter_leads
├── ✅ Completed: write_email
└── 🔄 Running: send_email

🌊 Flow: SalesPipeline
    ID: 591491fa-58d3-4833-83b6-fd14ccb348ed
├── Flow Method Step
├── ✅ Completed: fetch_leads
├── ✅ Completed: score_leads
├── ✅ Completed: store_leads_score
├── ✅ Completed: filter_leads
├── ✅ Completed: write_email
└── ✅ Completed: send_email

✅ Flow Finished: SalesPipeline
├── Flow Method Step
├── ✅ Completed: fetch_leads
├── ✅ Completed: score_leads
├── ✅ Completed: store_leads_score
├── ✅ Completed: filter_leads
├── ✅ Completed: write_email
└── ✅ Completed: send_email

╭──────────────────────────────────────────────── Flow Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Flow Execution Completed                                                                                       │
│  Name: SalesPipeline                                                                                            │
│  ID: 591491fa-58d3-4833-83b6-fd14ccb348ed                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Usage


In [48]:
import pandas as pd

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame(
    [flow.state["score_crews_results"][0].token_usage.dict()]
)

# Calculate total costs
costs = 0.150 * df_usage_metrics["total_tokens"].sum() / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Display the DataFrame
df_usage_metrics

Total costs: $0.0106


,total_tokens,prompt_tokens,cached_prompt_tokens,completion_tokens,successful_requests
0,70733,68786,0,1947,11


In [49]:
scores = flow.state["score_crews_results"]

## final scores


In [50]:
import pandas as pd
from IPython.display import display, HTML

lead_scoring_result = scores[0].pydantic

# Create a dictionary with the nested structure flattened
data = {
    "Name": lead_scoring_result.personal_info.name,
    "Job Title": lead_scoring_result.personal_info.job_title,
    "Role Relevance": lead_scoring_result.personal_info.role_relevance,
    "Professional Background": lead_scoring_result.personal_info.professional_background,
    "Company Name": lead_scoring_result.company_info.company_name,
    "Industry": lead_scoring_result.company_info.industry,
    "Company Size": lead_scoring_result.company_info.company_size,
    "Revenue": lead_scoring_result.company_info.revenue,
    "Market Presence": lead_scoring_result.company_info.market_presence,
    "Lead Score": lead_scoring_result.lead_score.score,
    "Scoring Criteria": ", ".join(lead_scoring_result.lead_score.scoring_criteria),
    "Validation Notes": lead_scoring_result.lead_score.validation_notes,
}

# Convert the dictionary to a DataFrame
df = pd.DataFrame.from_dict(data, orient="index", columns=["Value"])

# Reset the index to turn the original column names into a regular column
df = df.reset_index()

# Rename the index column to 'Attribute'
df = df.rename(columns={"index": "Attribute"})

# Create HTML table with bold attributes and left-aligned values
html_table = (
    df.style.set_properties(**{"text-align": "left"})
    .format({"Attribute": lambda x: f"<b>{x}</b>"})
    .hide(axis="index")
    .to_html()
)

# Display the styled HTML table
display(HTML(html_table))

Attribute,Value
Name,João Moura
Job Title,Director of Engineering
Role Relevance,10
Professional Background,"João Moura has over 20 years of experience in Software Engineering. He started his career in technology at an early age and held various significant positions like CTO at Palpiteros and now Director of Engineering at Clearbit. His experiences include managing multiple engineering teams and leading AI initiatives, emphasizing working remotely and building strong cross-functional teams."
Company Name,Clearbit
Industry,Technology
Company Size,8
Revenue,None
Market Presence,10
Lead Score,95


## emails


In [51]:
import textwrap

result_text = emails[0].raw
wrapped_text = textwrap.fill(result_text, width=80)
print(wrapped_text)

Thank you for your interest in CrewAI's Multi-Agent Orchestration Platform. With
your focus on agentic automation for enterprise-level operations, we believe our
platform can significantly enhance your process efficiency and innovation.  By
leveraging CrewAI, companies like yours can streamline complex workflows and
unlock new capabilities with precision. Our platform is specifically designed to
accommodate the dynamic needs of various verticals, facilitating seamless
integration and orchestration.  To see how CrewAI can be tailored to fit your
specific needs, please schedule a detailed walkthrough of our platform. This
meeting will provide insights into how our AI agents can be customized for your
business, ensuring you gain the maximum benefit from your investment.  [Schedule
a Walkthrough Now](#)  Don’t miss the opportunity to transform your enterprise
with cutting-edge AI technology. Act now to stay ahead in the automation curve.
Looking forward to showing you what CrewAI can achie

## How complex it can get


In [55]:
from crewai import Flow
from crewai.flow.flow import listen, start, and_, or_, router


class SalesPipeline(Flow):

    @start()
    def fetch_leads(self):
        # Pull our leads from the database
        # This is a mock, in a real-world scenario, this is where you would
        # fetch leads from a database
        leads = [
            {
                "lead_data": {
                    "name": "João Moura",
                    "job_title": "Director of Engineering",
                    "company": "Clearbit",
                    "email": "joao@clearbit.com",
                    "use_case": "Using AI Agent to do better data enrichment.",
                },
            },
        ]
        return leads

    @listen(fetch_leads)
    def score_leads(self, leads):
        scores = lead_scoring_crew.kickoff_for_each(leads)
        self.state["score_crews_results"] = scores
        return scores

    @listen(score_leads)
    def store_leads_score(self, scores):
        # Here we would store the scores in the database
        return scores

    @listen(score_leads)
    def filter_leads(self, scores):
        return [score for score in scores if score["lead_score"].score > 70]

    @listen(and_(filter_leads, store_leads_score))
    ## this will make sure it waits for filter_leads and store_leads_score to get done and only then it will get execute
    def log_leads(self, leads):
        print(f"Leads: {leads}")

    @router(filter_leads)
    def count_leads(self, scores):
        if len(scores) > 10:
            return "high"
        elif len(scores) > 5:
            return "medium"
        else:
            return "low"

    @listen("high")
    def store_in_salesforce(self, leads):
        return leads

    @listen("medium")
    def send_to_sales_team(self, leads):
        return leads

    @listen("low")
    def write_email(self, leads):
        scored_leads = [lead.to_dict() for lead in leads]
        emails = email_writing_crew.kickoff_for_each(scored_leads)
        return emails

    @listen(write_email)
    def send_email(self, emails):
        # Here we would send the emails to the leads
        return emails

In [56]:
flow = SalesPipeline()
flow.plot()

╭──────────────────────────────────────────────── Flow Execution ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Starting Flow Execution                                                                                        │
│  Name: SalesPipeline                                                                                            │
│  ID: 0ebc0b08-9c64-4a04-886a-8373854b5aac                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Plot saved as crewai_flow.html


In [57]:
import webbrowser
import os

path = os.path.abspath("crewai_flow.html")
webbrowser.open(f"file://{path}")

True